In [ ]:
%reset
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
# Stock market data
df = pd.read_csv("StockMarket.csv")

# Select variables
df = df[['ticker', 'year', 'month', 'ret', 'prc', 'shrout', 'mktcap', 'rf', 'mkt']]

print(df)

In [ ]:
###################
# QUESTION 1: APPLE

# Select the stock of Apple (ticker AAPL)
apple = df.loc[ (df['ticker']=="xxxxxxxxxx") & (df['year']==2019) & (df['month']==12), ['ticker', 'year', 'month', 'prc', 'shrout', 'mktcap','ret']]
print(apple)

In [ ]:
# Price, shares outstanding, and market cap
print( "Stock price (USD per share):", apple['xxxxxxxxxx'].iloc[0])
print( "Shares outstanding (billion):", apple['xxxxxxxxxx'].iloc[0]/1e6)
print( "Stock price x Shares outstanding (billion USD):", xxxxxxxxxx)
print( "Market capitalization (billion USD):", apple['xxxxxxxxxx'].iloc[0]/1e6)

In [ ]:
##################
# QUESTION 2: FORD

# Select the stock of Ford (ticker F)
df_i = df[df['ticker'] == "xxxxxxxxxx"].copy()
print(df_i[['ticker','year','month','ret']])

In [ ]:
# a. Average return of Ford
print( "Average return:", df_i['xxxxxxxxxx'].mean())

In [ ]:
# b. Stock return minus risk-free rate
df_i['retrf'] = df_i['xxxxxxxxxx'] - df_i['xxxxxxxxxx']
print( "Average return minus rf:", df_i['retrf'].mean())

In [ ]:
# c. Market return minus risk-free rate
df_i['mktrf'] = ??????????
print( "Average market return minus rf:", df_i['mktrf'].mean())

In [ ]:
# d. Estimate CAPM
y = xxxxxxxxxx
df_i['constant'] = 1
X = xxxxxxxxxx
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
#################################
# QUESTION 3: LONG-ONLY PORTFOLIO

# Group by year and month, and calculate the portfolio return
df_portfolio = df.groupby(['year', 'month', 'rf', 'mkt']).apply(
    lambda x: 0.25 * x.loc[x['ticker'] == 'F', 'ret'].values[0] + 
              xxxxxxxxxx * x.loc[x['ticker'] == 'GE', 'ret'].values[0] + 
              xxxxxxxxxx * x.loc[x['ticker'] == 'xxxxxxxxxx', 'ret'].values[0]
).reset_index(name='ret')

# Portfolio return
print(df_portfolio)

In [ ]:
# a. Mean portfolio return
print( "Mean portfolio return:", df_portfolio['ret'].mean())

In [ ]:
# b. Estimate CAPM
df_portfolio['constant'] = xxxxxxxxxx
df_portfolio['retrf'] = xxxxxxxxxx
df_portfolio['mktrf'] = xxxxxxxxxx
y = xxxxxxxxxx
X = xxxxxxxxxx
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
##################################
# QUESTION 4: LONG-SHORT PORTFOLIO

# Group by year and month, and calculate the portfolio return
df_portfolio = df.groupby(['year', 'month', 'rf', 'mkt']).apply(
    lambda x: ??????+or-?????? x['rf']
              ??????+or-?????? x.loc[x['ticker'] == 'F', 'ret'].values[0]
              ??????+or-?????? x.loc[x['ticker'] == 'GE', 'ret'].values[0]
).reset_index(name='ret')

In [ ]:
# a. Mean portfolio return
print( "Mean portfolio return:", df_portfolio['ret'].mean())

In [ ]:
# b. Estimate CAPM
df_portfolio['constant'] = 1
df_portfolio['retrf'] = df_portfolio['ret'] - df_portfolio['rf']
df_portfolio['mktrf'] = df_portfolio['mkt'] - df_portfolio['rf']
y = df_portfolio['retrf']
X = df_portfolio[['mktrf', 'constant']]
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
%reset
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
############################
# EXAMPLE: MOMENTUM STRATEGY

# Stock market data
df = pd.read_csv("StockMarket.csv")

# Select variables
df = df[['permno', 'ticker', 'year', 'month', 'ret', 'prc', 'shrout', 'mktcap', 'rf', 'mkt']]

# Sort the data by 'ticker', 'year', and 'month'
df = df.sort_values(by=['permno', 'year', 'month'])

# Step 1: Calculate cumulative return for each stock from month t-12 to t-2 (excluding t-1)
# Create return in t-2 by shift the return
df['ret_2'] = df.groupby('permno')['ret'].shift(2)

# Calculate cumulative return from t-12 to t-2
df['cum_ret'] = df.groupby('permno')['ret_2'].rolling(window=11, min_periods=11).apply(
    lambda x: np.prod(1 + x) - 1, raw=True).reset_index(level=0, drop=True)

# Check the result
print( df.loc[ (df['ticker']=="AAPL") & (df['year']<=1983), ['ticker', 'year', 'month', 'ret', 'cum_ret']])

In [ ]:
# Step 2: Rank stocks by their cumulative return for each month and year
# Drop rows with missing cumulative return values
df.dropna(subset=['cum_ret'], inplace=True)

# Rank stocks into deciles within each year and month
df['decile'] = df.groupby(['year', 'month'])['cum_ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# Check the result
print( df.loc[ (df['ticker']=="MSFT") & (df['year']==2000), ['ticker', 'year', 'month', 'ret', 'cum_ret', 'decile']])

In [ ]:
# Step 3: Select top decile for long and bottom decile for short
# Long: top decile (9), Short: bottom decile (0)
#df['position'] = 0
#df.loc[df['decile'] == 9, 'position'] = 1  # Long the top decile
#df.loc[df['decile'] == 0, 'position'] = -1  # Short the bottom decile
df['long'] = 0
df.loc[df['decile'] == 9, 'long'] = 1
df['short'] = 0
df.loc[df['decile'] == 0, 'short'] = 1

# Microsoft in 2018
print( df.loc[ (df['ticker']=="MSFT") & (df['year']==2018), ['ticker', 'year', 'month', 'ret', 'decile', 'long', 'short']])
# Ford in 2018
print( df.loc[ (df['ticker']=="F") & (df['year']==2018), ['ticker', 'year', 'month', 'ret', 'decile', 'long', 'short']])

In [ ]:
# Step 4: Define portfolio weights
# Define long portfolio weights such that they add up to 1
df['long_tot'] = df.groupby(['year', 'month'])['long'].transform('sum')
df['w_long'] = df['long'] / df['long_tot']
# Define short portfolio weights such that they add up to 1
df['short_tot'] = df.groupby(['year', 'month'])['short'].transform('sum')
df['w_short'] = df['short'] / df['short_tot']

# Portfolio weights on Microfost and Ford in 2018
print( df.loc[ (df['ticker']=="MSFT") & (df['year']==2018), ['ticker', 'year', 'month', 'ret', 'decile', 'w_long', 'w_short']])
print( df.loc[ (df['ticker']=="F") & (df['year']==2018), ['ticker', 'year', 'month', 'ret', 'decile', 'w_long', 'w_short']])

In [ ]:
# Step 5: Calculate returns for long-short portfolio
# Portfolio return = sum of return on short leg minus return on short leg
df['ret_p'] = df['ret'] * df['w_long'] - df['ret'] * df['w_short']
portfolio_ret = df.groupby(['year', 'month'])['ret_p'].sum().reset_index()

# The long-short portfolio return is the return of a zero-dollar portfolio, i.e., with total weights =0
# ==> Add the risk-free asset with weight 1 to the portfolio
market_ret = df.groupby(['year', 'month'])[['rf','mkt']].mean().reset_index()  # get risk-free rate and market return
portfolio_ret = pd.merge(portfolio_ret, market_ret, on=['year', 'month'], how='left') # merge
# Add the risk-free rate to the portfolio return
portfolio_ret['ret_p'] = portfolio_ret['ret_p'] + portfolio_ret['rf']

# Display the portfolio returns
print(portfolio_ret)

In [ ]:
# Step 6: Calculate the CAPM alpha of the momentum strategy
portfolio_ret['retrf_p'] = portfolio_ret['ret_p'] - portfolio_ret['rf']
y = portfolio_ret['retrf_p']
portfolio_ret['constant'] = 1
portfolio_ret['mktrf'] = portfolio_ret['mkt'] - portfolio_ret['rf']
X = portfolio_ret[['mktrf', 'constant']]
model = sm.OLS(y, X).fit()
print(model.summary())